<p style="background: linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F); /* Gradient Background */
          font-family: 'Roboto', sans-serif;
          color: #F39C12; /* White Text for Contrast */
          font-size: 1.9rem;
          text-align: center;
          padding: 20px 35px;
          border-radius: 40px;
          box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
          letter-spacing: 1px;">
    SMS CLASSIFIER : SPAM OR HAM
</p>



<p align="center">
  <img src="img.jpg" alt="image" style="width: 500px;"/>
</p>

   <a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">
<p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:150%;text-align:center;border-radius:20px 60px;">TABLE OF CONTENTS</p>   
    
* [1. Dataset](#1)
    
* [2. Importing Libraries](#2)
    
* [3. Loading Data](#3)
    
* [4. Visualization](#4)
     
* [5. Text Preprocessing](#5)
    
    * [5.1 Conversion to Lowercase](#5.1)
    * [5.2 Removing Urls](#5.2)
    * [5.3 Removing HTML Tags](#5.3)
    * [5.4 Removing Punctuation](#5.4)
    * [5.5 Replacing Shortconvo/Chatwords](#5.5)
    * [5.6 Spelling Correction](#5.6)
    * [5.7 Removing Stopwords](#5.7)
    * [5.8 Removing Emojis](#5.8)
    * [5.9 Lemmatization](#5.9)
    
* [6. Vectorization](#6) 
    * [6.1 Word2Vec](#6.1)
      
* [7. Data Preprocessing](#7)
    
* [8. Model Building](#8)
    
* [9. Model Evaluation](#9)


<a id="1"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Dataset</p>   
https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

<a id="2"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Importing Libraries</p>   


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from nltk.corpus import stopwords
import string
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report
import gensim.downloader as api
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess
import gensim
from sklearn.ensemble import RandomForestClassifier

<a id="3"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Loading Data</p>   


In [ ]:
data=pd.read_csv("SMSSpamCollection", delimiter="\t",names=["label","message"])
data.head()

<a id="4"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Visualization</p>   


In [ ]:
#Palette
cols= ["#F39C12", "#E598D8"] 
#first of all let us evaluate the target and find out if our data is imbalanced or not
plt.figure(figsize=(12,8))
fg = sns.countplot(x= data["label"], palette= cols)
fg.set_title("Count Plot of Classes", color="#58508d")
fg.set_xlabel("Classes", color="#58508d")
fg.set_ylabel("Number of Data points", color="#58508d")

<a id="5"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Text Preprocessing</p>   


<a id="5.1"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">1. Conversion to Lowercase</p>   


In [5]:
data['message'] = data['message'].str.lower()

<a id="5.2"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">2. Removing Urls</p>   


In [6]:
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)


In [7]:
data['message'] = data['message'].apply(remove_url)

<a id="5.3"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">3. Removing HTML Tags</p>   


In [8]:
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)



In [9]:
data['message'] = data['message'].apply(remove_html_tags)

<a id="5.4"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">4. Removing Punctuations</p>   


In [10]:
punct = string.punctuation
def remove_punct(text):
    return text.translate(str.maketrans('', '', punct))

In [11]:
data['message'] = data['message'].apply(remove_punct)

<a id="5.5"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">5. Replacing Short-Convo/Chat-Words</p>   


In [12]:
# Removing Short Convo / Chat Words
chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "B4N": "Bye For Now",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "ILU: I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don't care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "BFF": "Best friends forever",
    "CSL": "Can't stop laughing"
}

In [13]:
def chat_conversion(text):
    new_text = []
    for i in text.split():
        if i.upper() in chat_words:
            new_text.append(chat_words[i.upper()])
        else:
            new_text.append(i)
    return " ".join(new_text)

In [14]:
data['message'] = data['message'].apply(chat_conversion)

<a id="5.6"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">6. Spelling Correction</p>   


In [15]:

def correct_spell(text):
    return str(TextBlob(text).correct())


In [ ]:
data['message'] = data['message'].apply(correct_spell)

<a id="5.7"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">7. Removing Stopwords</p>   


In [17]:
stopword = set(stopwords.words('english'))
def remove_stopwords(text):
    new_text = []
    
    for word in text.split():
        if word in stopword:
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [18]:
data['message'] = data['message'].apply(remove_stopwords)

<a id="5.8"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">8. Removing Emojis</p>   


In [19]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [20]:
data['message'] = data['message'].apply(remove_emoji)

<a id="5.9"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">9. Lemmatization</p>   


In [21]:
lemmatizer = WordNetLemmatizer()

In [22]:
corpus = []
# just apply stemming i have already done text preprocessing
for i in range(0, len(data)):
    # Keep only alphabetic characters and split into words
    review = re.sub('[^a-zA-Z]', ' ', data['message'][i])
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in stopword]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
[[i, j, k] for i, j, k in zip(list(map(len, corpus)), corpus, data['message']) if i < 1]


<a id="6"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Tokenization</p>   


In [24]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [ ]:
words

<a id="6"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Vectorization</p>   


<a id="6.1"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Word2Vec</p>   



In [26]:
# Training Word2Vec Model from scratch
model=gensim.models.Word2Vec(words)


In [ ]:
model.wv.index_to_key

In [ ]:
model.corpus_count

In [ ]:
model.epochs

In [ ]:
model.wv.similar_by_word("good")

In [31]:
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)
    
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)
                #or [np.zeros(len(model.wv.index_to_key))], axis=0)

In [ ]:
from tqdm import tqdm
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

<a id="7"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Data Preprocessing</p>   


In [33]:
X_new = np.array(X,dtype=object)

In [34]:
y = data[list(map(lambda x: len(x) > 0, corpus))]
y = pd.get_dummies(y['label'])  # Convert labels to one-hot encoding

In [ ]:
# Initialize a list to store DataFrames
dfs = []

# Assuming X is defined (e.g., a list of numpy arrays)
for i in range(len(X)):
    dfs.append(pd.DataFrame(X[i].reshape(1, -1)))

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df.head()

In [ ]:
df['Output']=y

In [38]:
df.dropna(inplace=True)

In [39]:
X=df

<a id="8"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Model Building</p>   


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [42]:
classifier=RandomForestClassifier()
y_pred=classifier.predict(X_test)

<a id="9"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Model Evaluation</p>   


In [ ]:
from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(y_test,y_pred))

In [ ]:
print(classification_report(y_test,y_pred))